In [1]:
from api.db import db

await db.init()

In [4]:
res = await db.fetch("SELECT * FROM discount")
print(res)

shape: (4, 3)
┌─────────┬──────────────┬──────────────┐
│ id      ┆ multiplier   ┆ required_amt │
│ ---     ┆ ---          ┆ ---          │
│ i64     ┆ decimal[*,2] ┆ i64          │
╞═════════╪══════════════╪══════════════╡
│ 1870644 ┆ 0.50         ┆ 2            │
│ 1965488 ┆ 0.20         ┆ 4            │
│ 22334   ┆ 0.60         ┆ 10           │
│ 69420   ┆ 0.10         ┆ 3            │
└─────────┴──────────────┴──────────────┘


In [13]:
import polars as pl

# items = pl.read_csv("../db/csvs/item.csv")
items_to_insert = pl.DataFrame(
    {
        "item_id": [98333829],
        "list_id": [666],
        "quantity": [30],
    }
)

res = await db.insert(items_to_insert, "item_in_list", return_cols={"item_id"})
print(res)

shape: (1, 1)
┌──────────┐
│ item_id  │
│ ---      │
│ i64      │
╞══════════╡
│ 98333829 │
└──────────┘


In [6]:
res = await db.insert(items_to_insert, "item", pkey_cols={"id"}, return_cols={"id"})
print(res)

shape: (3, 1)
┌────────┐
│ id     │
│ ---    │
│ i64    │
╞════════╡
│ 394789 │
│ 3483   │
│ 39     │
└────────┘


In [11]:
res = await db.fetch(
    "DELETE FROM item WHERE id = ANY(:ids) RETURNING id", ids=[112233,394789,3483,39]
)
print(res)

shape: (4, 1)
┌────────┐
│ id     │
│ ---    │
│ i64    │
╞════════╡
│ 112233 │
│ 394789 │
│ 3483   │
│ 39     │
└────────┘


In [5]:
res = await db.fetch(
    "SELECT * FROM item WHERE id = ANY(:ids) ORDER BY price LIMIT 10",
    ids=[83473, 348374],
)
print(res)

shape: (2, 3)
┌────────┬─────────┬──────────────┐
│ id     ┆ name    ┆ price        │
│ ---    ┆ ---     ┆ ---          │
│ i64    ┆ str     ┆ decimal[*,2] │
╞════════╪═════════╪══════════════╡
│ 348374 ┆ yogurt  ┆ 3.99         │
│ 83473  ┆ pumpkin ┆ 12.99        │
└────────┴─────────┴──────────────┘


In [14]:
ids = pl.Series("id", [83473, 348374])
names = pl.Series("name", ["onion", "bbq", "beans"])
attr_dict = {"ids": ids, "names": names}

query = """--sql
    SELECT * 
    FROM item 
    WHERE 
        id = ANY(:ids) OR
        name = ANY(:names) 
    ORDER BY price 
    LIMIT 10
    """

await db.fetch(
    query,
    **attr_dict,
)

id,name,price
i64,str,"decimal[*,2]"
1965488,"""onion""",2.99
348374,"""yogurt""",3.99
1870644,"""bbq""",6.99
83473,"""pumpkin""",12.99
6334,"""beans""",69.99
